In [10]:
#step1: calculate video duration 

import cv2
import numpy as np
#from visualize_cv2 import model, display_instances, class_names




In [11]:

v = cv2.VideoCapture('test.mp4')
v.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
duration = v.get(cv2.CAP_PROP_POS_MSEC)

timestamp_each_frame = duration/60


print('duration (S) = ' + str(duration))
print('duration (S) = ' + str(timestamp_each_frame))

frameCount = int(v.get(cv2.CAP_PROP_FRAME_COUNT))
print('number of frames = ' + str(frameCount))

# the 1st frame is frame 0, not 1, so "5335" means after the last frame
POS_FRAMES = v.get(cv2.CAP_PROP_POS_FRAMES)
FRAME_COUNT = v.get(cv2.CAP_PROP_FRAME_COUNT)
print('POS_FRAMES = ' + str(POS_FRAMES))
print('FRAME_COUNT = ' + str(FRAME_COUNT))

v.release()

duration (S) = 8441.766666666666
duration (S) = 140.6961111111111
number of frames = 253
POS_FRAMES = 253.0
FRAME_COUNT = 253.0


In [5]:
#step2: output video frame time and frame content in a dictionary

In [7]:
import cv2
import numpy as np
from visualize_cv2 import model, display_instances, class_names
import json




In [26]:
capture = cv2.VideoCapture('test.mp4')
size = (
    int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
)
codec = cv2.VideoWriter_fourcc(*'DIVX')
output = cv2.VideoWriter('videofile_masked2.avi', codec, 1, size)

count = 0
out_dict = {}

while(capture.isOpened()):
    # cap.read() returns a bool (True/False). 
    # If frame is read correctly, it will be True. So you can check end of the video by checking this return value
    ret, frame = capture.read()
    if ret:
        # add mask to frame
        results = model.detect([frame], verbose=0)
        r = results[0]
        frame = display_instances(
            frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
        )
        output.write(frame)
        cv2.imshow('frame', frame)

        # put in dict
        # frame_excel = video length / 1/60 : r['class_ids']
        # CSV
        #https://www.programiz.com/python-programming/nested-dictionary
        #https://stackoverflow.com/questions/10756427/loop-through-all-nested-dictionary-values
        #output object: out_dict[count] = r['class_ids']
        #wrong: out_dict[count][r['class_ids']] = r['rois']
        #output bunding box: out_dict[count] = {}
        #for i, cls_id in enumerate(r['class_ids']):  
        #    out_dict[count][cls_id] = r['rois'][i, :]
        
        out_dict[count] = {}
        out_dict[count]['cls_id'] = r['class_ids']
        out_dict[count]['rois'] = r['rois']
        
        
        
 
        count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

capture.release()
output.release()
cv2.destroyAllWindows()


In [41]:
out_dict

{0: {'cls_id': array([ 3, 57, 14]), 'rois': array([[  4, 113,  74, 205],
         [ 15,  62,  46,  86],
         [107,   0, 238, 472]])},
 1: {'cls_id': array([ 3, 57, 14]), 'rois': array([[  3, 114,  75, 205],
         [ 15,  61,  46,  86],
         [ 98,   0, 235, 480]])},
 2: {'cls_id': array([ 3, 57]), 'rois': array([[  3, 114,  75, 205],
         [ 14,  61,  46,  87]])},
 3: {'cls_id': array([ 3, 57]), 'rois': array([[  4, 113,  74, 205],
         [ 15,  62,  46,  88]])},
 4: {'cls_id': array([ 3, 57, 14]), 'rois': array([[  4, 113,  74, 205],
         [ 15,  62,  46,  86],
         [104,   0, 237, 448]])},
 5: {'cls_id': array([ 3, 57,  8, 14]), 'rois': array([[  5, 113,  74, 204],
         [ 15,  62,  46,  86],
         [ 17, 234,  73, 306],
         [106,   2, 234, 444]])},
 6: {'cls_id': array([ 3, 57, 14]), 'rois': array([[  4, 113,  74, 205],
         [ 15,  62,  46,  86],
         [106,   2, 234, 443]])},
 7: {'cls_id': array([ 3, 57, 14,  8]), 'rois': array([[  5, 113,  74

In [42]:
each_time_interval = duration / (frameCount-1)

In [43]:
import collections
frame2content = {}
for k, v in out_dict.items():
    objs = [class_names[i] for i in v["cls_id"]]
    obj2num = dict(collections.Counter(objs))
    _objs = []
    for o in objs:
        _objs.append(o + "_" + str(obj2num[o]))
        obj2num[o] -= 1
        
    obj2roi = {}
    for idx, o in enumerate(_objs):
        obj2roi[o] = list(map(int, list(v["rois"][idx, :])))
        #obj2roi[o] = v["rois"][idx, :]
    frame2content[k * each_time_interval] = obj2roi
        
    

In [44]:
print(frame2content)

{0.0: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [107, 0, 238, 472]}, 33.49907407407407: {'car_1': [3, 114, 75, 205], 'chair_1': [15, 61, 46, 86], 'bench_1': [98, 0, 235, 480]}, 66.99814814814815: {'car_1': [3, 114, 75, 205], 'chair_1': [14, 61, 46, 87]}, 100.49722222222222: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 88]}, 133.9962962962963: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [104, 0, 237, 448]}, 167.49537037037038: {'car_1': [5, 113, 74, 204], 'chair_1': [15, 62, 46, 86], 'truck_1': [17, 234, 73, 306], 'bench_1': [106, 2, 234, 444]}, 200.99444444444444: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [106, 2, 234, 443]}, 234.4935185185185: {'car_1': [5, 113, 74, 204], 'chair_1': [15, 61, 47, 87], 'bench_1': [111, 0, 230, 453], 'truck_1': [17, 234, 73, 306]}, 267.9925925925926: {'car_1': [3, 114, 75, 205], 'chair_1': [15, 62, 45, 86], 'bench_1': [104, 0, 234, 475]}, 301.4916666666667: {'car_2': [

In [45]:
import json
with open('nik2.json', 'w') as fp:
   json.dump(frame2content, fp, indent=2)

In [ ]:
#step3: convert dictionary to CSV file

In [14]:
#with open('frames.json', 'w') as fp:
#   json.dump(out_dict, fp, indent=2)

In [12]:
out_dict

{0: {3: array([  4, 113,  74, 205]),
  57: array([15, 62, 46, 86]),
  14: array([107,   0, 238, 472])},
 1: {3: array([  3, 114,  75, 205]),
  57: array([15, 61, 46, 86]),
  14: array([ 98,   0, 235, 480])},
 2: {3: array([  3, 114,  75, 205]), 57: array([14, 61, 46, 87])},
 3: {3: array([  4, 113,  74, 205]), 57: array([15, 62, 46, 88])},
 4: {3: array([  4, 113,  74, 205]),
  57: array([15, 62, 46, 86]),
  14: array([104,   0, 237, 448])},
 5: {3: array([  5, 113,  74, 204]),
  57: array([15, 62, 46, 86]),
  8: array([ 17, 234,  73, 306]),
  14: array([106,   2, 234, 444])},
 6: {3: array([  4, 113,  74, 205]),
  57: array([15, 62, 46, 86]),
  14: array([106,   2, 234, 443])},
 7: {3: array([  5, 113,  74, 204]),
  57: array([15, 61, 47, 87]),
  14: array([111,   0, 230, 453]),
  8: array([ 17, 234,  73, 306])},
 8: {3: array([  3, 114,  75, 205]),
  57: array([15, 62, 45, 86]),
  14: array([104,   0, 234, 475])},
 9: {3: array([ 16, 236,  71, 306]),
  57: array([14, 63, 45, 87]),
  

In [13]:
each_time_interval = duration / (frameCount-1)
each_time_interval

33.49907407407407

In [14]:
for key in out_dict.keys():
    each_frame_in_millseconds = key * each_time_interval
    print(each_frame_in_millseconds)
#each_frame_in_millseconds

0.0
33.49907407407407
66.99814814814815
100.49722222222222
133.9962962962963
167.49537037037038
200.99444444444444
234.4935185185185
267.9925925925926
301.4916666666667
334.99074074074076
368.4898148148148
401.9888888888889
435.48796296296297
468.987037037037
502.4861111111111
535.9851851851852
569.4842592592593
602.9833333333333
636.4824074074074
669.9814814814815
703.4805555555555
736.9796296296296
770.4787037037037
803.9777777777778
837.4768518518518
870.9759259259259
904.475
937.974074074074
971.4731481481481
1004.9722222222222
1038.4712962962963
1071.9703703703703
1105.4694444444444
1138.9685185185185
1172.4675925925926
1205.9666666666667
1239.4657407407408
1272.9648148148149
1306.463888888889
1339.962962962963
1373.462037037037
1406.961111111111
1440.460185185185
1473.9592592592592
1507.4583333333333
1540.9574074074073
1574.4564814814814
1607.9555555555555
1641.4546296296296
1674.9537037037037
1708.4527777777778
1741.9518518518519
1775.450925925926
1808.95
1842.4490740740741
1875

In [15]:
#frame_in_millseconds = list(out_dict.keys())*each_time_interval

In [16]:
#for key in out_dict.keys():
#    each_frame_in_millseconds = key * each_time_interval
#    even_num_to_square = {each_frame_in_millseconds : x ** 2 for x in nums if x % 2 == 0}
#    print(even_num_to_square) 


In [17]:
#each_frame_content_number = out_dict.values()
#each_frame_content_number

In [18]:
for value in out_dict.values():
    each_frame_content_numbers = value
    print(each_frame_content_numbers)
#each_frame_content_number = out_dict.values()
#each_frame_content_number

{3: array([  4, 113,  74, 205]), 57: array([15, 62, 46, 86]), 14: array([107,   0, 238, 472])}
{3: array([  3, 114,  75, 205]), 57: array([15, 61, 46, 86]), 14: array([ 98,   0, 235, 480])}
{3: array([  3, 114,  75, 205]), 57: array([14, 61, 46, 87])}
{3: array([  4, 113,  74, 205]), 57: array([15, 62, 46, 88])}
{3: array([  4, 113,  74, 205]), 57: array([15, 62, 46, 86]), 14: array([104,   0, 237, 448])}
{3: array([  5, 113,  74, 204]), 57: array([15, 62, 46, 86]), 8: array([ 17, 234,  73, 306]), 14: array([106,   2, 234, 444])}
{3: array([  4, 113,  74, 205]), 57: array([15, 62, 46, 86]), 14: array([106,   2, 234, 443])}
{3: array([  5, 113,  74, 204]), 57: array([15, 61, 47, 87]), 14: array([111,   0, 230, 453]), 8: array([ 17, 234,  73, 306])}
{3: array([  3, 114,  75, 205]), 57: array([15, 62, 45, 86]), 14: array([104,   0, 234, 475])}
{3: array([ 16, 236,  71, 306]), 57: array([14, 63, 45, 87]), 14: array([109,   0, 238, 474])}
{3: array([ 16, 236,  70, 306]), 57: array([14, 63, 

{1: array([ 77,   0, 262, 238]), 57: array([15, 62, 46, 87]), 3: array([ 17, 233,  75, 308]), 17: array([ 51, 202,  95, 235]), 68: array([ 98,   0, 114,   8])}
{1: array([ 42,   0, 270, 236]), 57: array([15, 62, 46, 87]), 3: array([ 16, 233,  75, 308])}
{1: array([ 52,   7, 273, 234]), 3: array([ 17, 235,  75, 308]), 57: array([15, 62, 46, 87])}
{3: array([ 18, 236,  75, 308]), 57: array([15, 62, 46, 87]), 1: array([ 90,  13, 283, 232])}
{1: array([145,   3, 320, 219]), 57: array([15, 62, 46, 87]), 3: array([ 18, 236,  75, 308])}
{57: array([15, 62, 46, 87]), 3: array([ 18, 236,  74, 307]), 1: array([177,   2, 352, 212])}
{57: array([15, 62, 47, 87]), 3: array([ 17, 235,  75, 308]), 1: array([186,   0, 347, 195]), 17: array([212,   6, 352, 193])}
{1: array([214,   0, 345, 195]), 3: array([ 17, 235,  74, 308]), 57: array([15, 63, 47, 88])}
{1: array([225,   0, 342, 192]), 3: array([ 16, 236,  73, 307]), 57: array([16, 62, 47, 87])}
{1: array([232,   0, 342, 192]), 3: array([ 16, 236,  7

In [ ]:
frame2content2position = {}
for k, v in out_dict.items():
    for i, j in v.items():
        content = class_names[i]
        print(content)
    #frame2content2position[k * each_time_interval][content] = r['rois']   

In [ ]:
        out_dict[count] = {}
        
        for i, cls_id in enumerate(r['class_ids']):  
            out_dict[count][cls_id] = r['rois'][i, :]

In [22]:
for k, v in out_dict.items():
   # print(v)
    for i, j in v.items():
        print(i)

3
57
14
3
57
14
3
57
3
57
3
57
14
3
57
8
14
3
57
14
3
57
14
8
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
3
57
14
3
57
14
3
57
14
3
57
3
57
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
57
3
14
57
3
14
3
57
14
3
57
14
57
3
14
3
57
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
3
57
14
3
57
14
3
57
14
57
3
14
3
57
14
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
3
57
14
3
57
3
57
3
57
3
57
3
57
57
3
57
3
57
3
57
3
57
3
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
57
3
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
3
57
14
14
57
3
57
3
14
14
3
57
3
57
14
3
57
14
3

In [25]:
frame2content = {}
for k, v in out_dict.items():
    content = [class_names[i] for i in v]
    print(content)
    
    
    out_dict[count] = {}    
    for j, cls_idd in enumerate(content):  
        out_dict[count][cls_idd] = r['rois'][j, :]

        
                out_dict[count] = {}
        
        for i, cls_id in enumerate(r['class_ids']):  
            out_dict[count][cls_id] = r['rois'][i, :]
    
    frame2content = {}
for k, v in out_dict.items():
    content = [class_names[i] for i in v]
    print(content)
    frame2content[k * each_time_interval] = content
frame2content
    #frame2content[k * each_time_interval] = content
#frame2content  

['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair']
['car', 'chair']
['car', 'chair', 'bench']
['car', 'chair', 'truck', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench', 'truck']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair']
['car', 'chair']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chai

{0.0: ['car', 'chair', 'bench'],
 33.49907407407407: ['car', 'chair', 'bench'],
 66.99814814814815: ['car', 'chair'],
 100.49722222222222: ['car', 'chair'],
 133.9962962962963: ['car', 'chair', 'bench'],
 167.49537037037038: ['car', 'chair', 'truck', 'bench'],
 200.99444444444444: ['car', 'chair', 'bench'],
 234.4935185185185: ['car', 'chair', 'bench', 'truck'],
 267.9925925925926: ['car', 'chair', 'bench'],
 301.4916666666667: ['car', 'chair', 'bench'],
 334.99074074074076: ['car', 'chair', 'bench'],
 368.4898148148148: ['car', 'chair', 'bench'],
 401.9888888888889: ['car', 'chair', 'bench'],
 435.48796296296297: ['car', 'chair', 'bench'],
 468.987037037037: ['car', 'chair', 'bench'],
 502.4861111111111: ['car', 'chair'],
 535.9851851851852: ['car', 'chair', 'bench'],
 569.4842592592593: ['car', 'chair', 'bench'],
 602.9833333333333: ['car', 'chair', 'bench'],
 636.4824074074074: ['car', 'chair'],
 669.9814814814815: ['car', 'chair'],
 703.4805555555555: ['car', 'chair', 'bench'],
 73

In [38]:
import collections
frame2content = {}
for k, v in out_dict.items():
    objs = [class_names[i] for i in v["cls_id"]]
    obj2num = dict(collections.Counter(objs))
    _objs = []
    for o in objs:
        _objs.append(o + "_" + str(obj2num[o]))
        obj2num[o] -= 1
        
    obj2roi = {}
    for idx, o in enumerate(_objs):
        obj2roi[o] = list(map(int, list(v["rois"][idx, :])))
        #obj2roi[o] = v["rois"][idx, :]
    frame2content[k * each_time_interval] = obj2roi
        
    

In [39]:
print(frame2content)

{0.0: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [107, 0, 238, 472]}, 33.49907407407407: {'car_1': [3, 114, 75, 205], 'chair_1': [15, 61, 46, 86], 'bench_1': [98, 0, 235, 480]}, 66.99814814814815: {'car_1': [3, 114, 75, 205], 'chair_1': [14, 61, 46, 87]}, 100.49722222222222: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 88]}, 133.9962962962963: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [104, 0, 237, 448]}, 167.49537037037038: {'car_1': [5, 113, 74, 204], 'chair_1': [15, 62, 46, 86], 'truck_1': [17, 234, 73, 306], 'bench_1': [106, 2, 234, 444]}, 200.99444444444444: {'car_1': [4, 113, 74, 205], 'chair_1': [15, 62, 46, 86], 'bench_1': [106, 2, 234, 443]}, 234.4935185185185: {'car_1': [5, 113, 74, 204], 'chair_1': [15, 61, 47, 87], 'bench_1': [111, 0, 230, 453], 'truck_1': [17, 234, 73, 306]}, 267.9925925925926: {'car_1': [3, 114, 75, 205], 'chair_1': [15, 62, 45, 86], 'bench_1': [104, 0, 234, 475]}, 301.4916666666667: {'car_2': [

In [40]:
import json
with open('nik.json', 'w') as fp:
   json.dump(frame2content, fp, indent=2)

In [92]:
frame2content = {}
for k, v in out_dict.items():
    content = [class_names[i] for i in v]
    print(content)
    frame2content[k * each_time_interval] = content
    

['car', 'chair', 'bench']
['car', 'chair', 'bench']
['car', 'chair']
['car', 'chair']
['car', 'chair', 'bench']
['car', 'chair', 'truck', 'bench']
['car', 'chair', 'bench']
['car', 'chair', 'bench', 'truck']
['car', 'chair', 'bench']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench']
['car', 'chair', 'car', 'bench']
['car', 'chair', 'car']
['car', 'chair', 'car', 'bench']
['car', 'chair', 'car', 'bench']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'car']
['car', 'chair', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair', 'bench', 'car']
['car', 'chair'

In [93]:
frame2content

{0.0: ['car', 'chair', 'bench'],
 33.49907407407407: ['car', 'chair', 'bench'],
 66.99814814814815: ['car', 'chair'],
 100.49722222222222: ['car', 'chair'],
 133.9962962962963: ['car', 'chair', 'bench'],
 167.49537037037038: ['car', 'chair', 'truck', 'bench'],
 200.99444444444444: ['car', 'chair', 'bench'],
 234.4935185185185: ['car', 'chair', 'bench', 'truck'],
 267.9925925925926: ['car', 'chair', 'bench'],
 301.4916666666667: ['car', 'chair', 'bench', 'car'],
 334.99074074074076: ['car', 'chair', 'bench', 'car'],
 368.4898148148148: ['car', 'chair', 'bench', 'car'],
 401.9888888888889: ['car', 'chair', 'bench', 'car'],
 435.48796296296297: ['car', 'chair', 'bench'],
 468.987037037037: ['car', 'chair', 'car', 'bench'],
 502.4861111111111: ['car', 'chair', 'car'],
 535.9851851851852: ['car', 'chair', 'car', 'bench'],
 569.4842592592593: ['car', 'chair', 'car', 'bench'],
 602.9833333333333: ['car', 'chair', 'bench', 'car'],
 636.4824074074074: ['car', 'chair', 'car'],
 669.9814814814815

In [5]:
each_time_interval = 33.49907407407407
out_dict = {1: {3: [[ 15,  62,  47,  88],[  5, 112,  74, 206],[ 15, 237,  72, 307]], 
                5: [[ 15,  62,  47,  88],[  5, 112,  74, 206],[ 15, 237,  72, 307]], 
                8: [[ 15,  62,  47,  88],[  5, 112,  74, 206],[ 15, 237,  72, 307]]},
           2: { 8: [[ 15,  62,  47,  88],[  5, 112,  74, 206],[ 15, 237,  72, 307]], 
                9: [[ 15,  62,  47,  88],[  5, 112,  74, 206],[ 15, 237,  72, 307]], 
                2: [[ 15,  62,  47,  88],[  5, 112,  74, 206],[ 15, 237,  72, 307]]}}



frame2content2position = {}
for k, v in out_dict.items():
    for i, j in v.items():
        content = class_names[i]
        print(content)
    #frame2content2position[k * each_time_interval][content] = r['rois']   

    

car
airplane
truck


NameError: name 'r' is not defined

In [ ]:
/
        
    content = [class_names[i] for i in v]
    print(content)
    frame2content2position[k * each_time_interval] = content
    
    
            #out_dict[count] = r['class_ids']
        out_dict[count] = {}
        
        out_dict[count][r['class_ids']] = r['rois']

In [98]:
import pandas as pd
import numpy as np
import csv


In [102]:
#method 1 to convert dict to CSV
#my_dict = dict( A = np.array([1,2]), B = np.array([1,2,3,4]) )
df1 = pd.DataFrame.from_dict(frame2content, orient='index')
df1
#df1.transpose()

,0,1,2,3,4,5
0.000000,car,chair,bench,None,None,None
33.499074,car,chair,bench,None,None,None
66.998148,car,chair,None,None,None,None
100.497222,car,chair,None,None,None,None
133.996296,car,chair,bench,None,None,None
167.495370,car,chair,truck,bench,None,None
200.994444,car,chair,bench,None,None,None
234.493519,car,chair,bench,truck,None,None
267.992593,car,chair,bench,None,None,None
301.491667,car,chair,bench,car,None,None


In [104]:
df1.to_csv('frame_test_video2.csv')

In [100]:
#method 2 to convert dict to CSV
#mydict = {'one': [1,2,3], 2: [4,5,6,7], 3: 8}

dict_df = pd.DataFrame({ key:pd.Series(value) for key, value in frame2content.items() })

dict_df
    

,0.0,33.49907407407407,66.99814814814815,100.49722222222222,133.9962962962963,167.49537037037038,200.99444444444444,234.4935185185185,267.9925925925926,301.4916666666667,...,8140.275,8173.774074074074,8207.273148148148,8240.772222222222,8274.271296296296,8307.77037037037,8341.269444444444,8374.768518518518,8408.267592592592,8441.766666666666
0,car,car,car,car,car,car,car,car,car,car,...,person,person,person,person,person,chair,chair,chair,chair,chair
1,chair,chair,chair,chair,chair,chair,chair,chair,chair,chair,...,chair,chair,chair,chair,chair,person,person,person,car,car
2,bench,bench,NaN,NaN,bench,truck,bench,bench,bench,bench,...,car,car,car,car,car,car,car,car,person,person
3,NaN,NaN,NaN,NaN,NaN,bench,NaN,truck,NaN,car,...,baseball glove,baseball glove,baseball glove,car,baseball glove,baseball glove,baseball glove,baseball glove,baseball glove,baseball glove
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,car,car,baseball glove,car,car,car,car,car,car
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,bench,bench,bench,bench,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
dict_df.to_csv('frame_test_video.csv')

In [ ]:
#reference
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.from_dict.html
#https://stackoverflow.com/questions/19736080/creating-dataframe-from-a-dictionary-where-entries-have-different-lengths

In [106]:
frame

In [105]:
r['rois']

array([[ 15,  62,  47,  88],
       [  5, 112,  74, 206],
       [277,   1, 352, 201],
       [286, 161, 325, 205],
       [ 15, 237,  72, 307]])

In [4]:
a = np.array([[ 15,  62,  47,  88],
       [  5, 112,  74, 206],
       [277,   1, 352, 201],
       [286, 161, 325, 205],
       [ 15, 237,  72, 307]])
print(a.shape)

(5, 4)


In [107]:
r['masks']

array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]

In [108]:
r['class_ids']

array([57,  3,  1, 36,  3])

In [109]:
class_names

['BG',
 'person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [110]:
r['scores']

array([0.98131555, 0.9773521 , 0.9694858 , 0.93024623, 0.7606261 ],
      dtype=float32)